# Day 1
For the QBs, RBs, and WRs, find the top three seasons by average fantasy points scored per player (standard scoring).  
Then find the player with the highest score per position-season combination.

In [1]:
import pandas as pd
from os import path
import sqlite3
import nfl_data_py as nfl


# Create database connection
conn = sqlite3.connect('../data/db/database.db')

In [2]:
query = """
    WITH position_season_data AS (
        SELECT
            position,
            s.season,
            ROUND(AVG(fantasy_points),2) AS avg_standard
        FROM seasonal s
        INNER JOIN (
            SELECT
                player_id,
                position
            FROM rosters
            WHERE position IN ('QB', 'WR', 'RB')
        ) AS roster_filtered
            ON roster_filtered.player_id = s.player_id
        WHERE season_type = 'REG'
        GROUP BY position, s.season
    ),
    position_season_rankings AS (
        SELECT
            position,
            season,
            avg_standard,
            -- For each position, rank the seasons by average fantasy points (standard)
            RANK() OVER(PARTITION BY position ORDER BY avg_standard DESC) AS rank_standard
        FROM position_season_data
    ),
    player_season_data AS (
        SELECT
            player_name,
            position,
            s.season,
            fantasy_points,
            -- For each season, per positon, rank the players by total fantasy points
            RANK() OVER(PARTITION BY position, season ORDER BY fantasy_points DESC) AS rank_standard
        FROM seasonal s
        INNER JOIN (
            SELECT Distinct
                player_id,
                player_name,
                position
            FROM rosters
            WHERE position IN ('QB', 'WR', 'RB')
        ) AS roster_filtered
            ON roster_filtered.player_id = s.player_id
        WHERE season_type = 'REG'
    )
    SELECT
        pos_sr.position,
        pos_sr.season,
        pos_sr.rank_standard AS r_avg_fantasy_pts,
        play_sr.player_name AS top_player
    FROM position_season_rankings AS pos_sr
    LEFT JOIN (
        SELECT player_name, position, season
        FROM player_season_data
        -- Only return the player with the highest fantasy points for that season
        WHERE rank_standard = 1) AS play_sr
        ON play_sr.position = pos_sr.position
            AND play_sr.season = pos_sr.season
    WHERE pos_sr.rank_standard <= 3;
"""

pd.read_sql(query, conn)

,position,season,r_avg_fantasy_pts,top_player
0,QB,2013,1,Peyton Manning
1,QB,2015,2,Cam Newton
2,QB,2016,3,Aaron Rodgers
3,RB,2006,1,LaDainian Tomlinson
4,RB,2008,2,DeAngelo Williams
5,RB,2009,3,Chris Johnson
6,WR,2002,1,Marvin Harrison
7,WR,2007,2,Randy Moss
8,WR,2013,3,Josh Gordon
